In [1]:
from bs4 import BeautifulSoup 
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver import ActionChains
import lxml
import time 
import re 
import pandas as pd
import numpy as np
from multiprocessing import Pool
import multiprocessing
import json
import urllib.request
from urllib.request import urlretrieve
import concurrent.futures
import os
print(os.getcwd())

C:\Users\HyunJu Kim\Desktop\졸작


In [2]:
def getPageLinksWantRange(startPageNo, lastPageNo): 
    links = []

    for pageNo in range(startPageNo, lastPageNo):
        url = "http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=09&PageNumber=" + str(pageNo + 1) 
        req = requests.get(url)
        soup = BeautifulSoup(req.content, "lxml") 
        booklinks = soup.select(('td.goodsTxtInfo p.review a')) 

        for booklink in booklinks: 
            link = str(booklink.get('href')).replace('#Review','')
            reviewP = 'http://www.yes24.com/Product/communityModules/GoodsReviewList/'+link[15:]+'?Sort=1&PageNumber='
            bookP = "http://www.yes24.com"+link
            links.append([reviewP,bookP])
    return links

In [3]:
def getBooksInfo(links):
    chrome = wd.Chrome()
    book_title = "h2.gd_name"
    book_topic = "dd ul.yesAlertLi a"
    book_review = "div.reviewInfoBot.origin div.review_cont"
    book_isbn = "td.txt.lastCol"
    btn_next = "a.bgYUI.next"

    global book_list

    for t, (rp, bp) in enumerate(links):
        print(t,'    ',bp)
        

        try:
            chrome.get(bp)
            chrome.implicitly_wait(3)
            local = pd.DataFrame()
            reviews = ''

            title = chrome.find_element_by_css_selector(book_title).text
            topic = chrome.find_elements_by_css_selector(book_topic)[1].text
            isbn = chrome.find_elements_by_css_selector(book_isbn)[2].text

            local['title'] = [title]
            local['topic'] = [topic]
            local['isbn'] = [isbn]
            
#             print(local['isbn'])
#             print(local['title'])
#             print(local['topic'])
            
            for i in range(1,11):
                review_url = rp + str(i)

                chrome.get(review_url)
                r = chrome.find_elements_by_css_selector(book_review)

                for j in range(len(r)):
                    reviews += r[j].text

            local['review'] = reviews.replace('\n','')
            book_list = book_list.append(local,ignore_index=True)
            
#             if not os.path.exists('yes24Img'):
#                 os.mkdir('yes24Img')
#             getImg('yes24 '+title, isbn)

        except:
            pass

In [4]:
yes24_links = getPageLinksWantRange(0,50)
print(len(yes24_links))
# print(yes24_links)

927


In [5]:
# workers = multiprocessing.cpu_count()*2
book_list = pd.DataFrame()

start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.submit(getBooksInfo, yes24_links[:100])
    executor.submit(getBooksInfo, yes24_links[100:200])
    executor.submit(getBooksInfo, yes24_links[200:300])
    executor.submit(getBooksInfo, yes24_links[300:400])
    executor.submit(getBooksInfo, yes24_links[400:500])
    executor.submit(getBooksInfo, yes24_links[500:600])
    executor.submit(getBooksInfo, yes24_links[600:700])
    executor.submit(getBooksInfo, yes24_links[700:800])
    executor.submit(getBooksInfo, yes24_links[800:900])
    executor.submit(getBooksInfo, yes24_links[900:])
print("time :", time.time() - start)

0      http://www.yes24.com/Product/Goods/90593181
0      http://www.yes24.com/Product/Goods/90686937
0      http://www.yes24.com/Product/Goods/15748042
0      http://www.yes24.com/Product/Goods/25283908
0      http://www.yes24.com/Product/Goods/90309256
0      http://www.yes24.com/Product/Goods/85117432
0      http://www.yes24.com/Product/Goods/74602725
00      http://www.yes24.com/Product/Goods/85434038
      http://www.yes24.com/Product/Goods/12981014
0      http://www.yes24.com/Product/Goods/89381473
1      http://www.yes24.com/Product/Goods/90960146
1      http://www.yes24.com/Product/Goods/89309569
1      http://www.yes24.com/Product/Goods/85466721
1      http://www.yes24.com/Product/Goods/30548474
1      http://www.yes24.com/Product/Goods/91002713
1      http://www.yes24.com/Product/Goods/91358628
1      http://www.yes24.com/Product/Goods/21631904
1      http://www.yes24.com/Product/Goods/69411597
1      http://www.yes24.com/Product/Goods/87661778
1      http://www.yes24.com/Pro

17      http://www.yes24.com/Product/Goods/39441749
16      http://www.yes24.com/Product/Goods/70921212
15      http://www.yes24.com/Product/Goods/91065135
18      http://www.yes24.com/Product/Goods/84807166
17      http://www.yes24.com/Product/Goods/90248133
1615      http://www.yes24.com/Product/Goods/91117619
      http://www.yes24.com/Product/Goods/90866098
16      http://www.yes24.com/Product/Goods/89404482
18      http://www.yes24.com/Product/Goods/90926435
16      http://www.yes24.com/Product/Goods/29646280
19      http://www.yes24.com/Product/Goods/91004966
17      http://www.yes24.com/Product/Goods/3352480
17      http://www.yes24.com/Product/Goods/75235476
18      http://www.yes24.com/Product/Goods/84768485
19      http://www.yes24.com/Product/Goods/90473307
17      http://www.yes24.com/Product/Goods/12901038
14      http://www.yes24.com/Product/Goods/37204310
17      http://www.yes24.com/Product/Goods/91053406
16      http://www.yes24.com/Product/Goods/90686903
18      http:

33      http://www.yes24.com/Product/Goods/90889308
35      http://www.yes24.com/Product/Goods/91124891
33      http://www.yes24.com/Product/Goods/22793441
29      http://www.yes24.com/Product/Goods/40204017
31      32http://www.yes24.com/Product/Goods/91207379
      http://www.yes24.com/Product/Goods/91072591
36      http://www.yes24.com/Product/Goods/91167039
34      http://www.yes24.com/Product/Goods/232775
34      http://www.yes24.com/Product/Goods/90703124
33      http://www.yes24.com/Product/Goods/88372513
35      http://www.yes24.com/Product/Goods/90128641
36      http://www.yes24.com/Product/Goods/63038113
30      http://www.yes24.com/Product/Goods/3142428
37      http://www.yes24.com/Product/Goods/91129988
37      http://www.yes24.com/Product/Goods/89938917
31      http://www.yes24.com/Product/Goods/87113038
32      http://www.yes24.com/Product/Goods/90624430
33      http://www.yes24.com/Product/Goods/84936246
35      http://www.yes24.com/Product/Goods/91112501
29      http://

53      http://www.yes24.com/Product/Goods/90406595
48      http://www.yes24.com/Product/Goods/79255315
44      http://www.yes24.com/Product/Goods/8157957
51      http://www.yes24.com/Product/Goods/37525733
51      http://www.yes24.com/Product/Goods/90428075
48      http://www.yes24.com/Product/Goods/90365124
46      http://www.yes24.com/Product/Goods/89906619
56      http://www.yes24.com/Product/Goods/90554248
49      http://www.yes24.com/Product/Goods/89990128
52      http://www.yes24.com/Product/Goods/90548633
45      http://www.yes24.com/Product/Goods/90635447
52      http://www.yes24.com/Product/Goods/17199481
54      http://www.yes24.com/Product/Goods/83544448
49      http://www.yes24.com/Product/Goods/61178234
52      http://www.yes24.com/Product/Goods/90396662
53      http://www.yes24.com/Product/Goods/64480601
57      http://www.yes24.com/Product/Goods/85028097
50      http://www.yes24.com/Product/Goods/89958114
47      http://www.yes24.com/Product/Goods/91070368
46      http:

68      http://www.yes24.com/Product/Goods/90966222
65      http://www.yes24.com/Product/Goods/86006021
61      http://www.yes24.com/Product/Goods/74372342
74      http://www.yes24.com/Product/Goods/85028105
68      http://www.yes24.com/Product/Goods/90977275
70      http://www.yes24.com/Product/Goods/85371532
70      http://www.yes24.com/Product/Goods/89326029
67      http://www.yes24.com/Product/Goods/61778623
64      http://www.yes24.com/Product/Goods/82589741
68      http://www.yes24.com/Product/Goods/13016823
71      http://www.yes24.com/Product/Goods/90125491
69      http://www.yes24.com/Product/Goods/82935976
71      http://www.yes24.com/Product/Goods/90959576
62      http://www.yes24.com/Product/Goods/90860768
66      http://www.yes24.com/Product/Goods/57613730
69      http://www.yes24.com/Product/Goods/90978496
75      http://www.yes24.com/Product/Goods/74261416
65      http://www.yes24.com/Product/Goods/91085744
76      http://www.yes24.com/Product/Goods/86017987
69      http

82      http://www.yes24.com/Product/Goods/30872852
79      http://www.yes24.com/Product/Goods/67449477
85      http://www.yes24.com/Product/Goods/90964078
84      http://www.yes24.com/Product/Goods/90862296
86      http://www.yes24.com/Product/Goods/90629220
91      http://www.yes24.com/Product/Goods/3257753
88      http://www.yes24.com/Product/Goods/74257925
76      http://www.yes24.com/Product/Goods/17245030
83      http://www.yes24.com/Product/Goods/3224857
94      http://www.yes24.com/Product/Goods/89849970
80      http://www.yes24.com/Product/Goods/69655504
85      http://www.yes24.com/Product/Goods/90765128
86      http://www.yes24.com/Product/Goods/90582727
92      http://www.yes24.com/Product/Goods/53802600
77      http://www.yes24.com/Product/Goods/11681152
84      http://www.yes24.com/Product/Goods/91165789
87      http://www.yes24.com/Product/Goods/89903225
81      http://www.yes24.com/Product/Goods/87116793
93      http://www.yes24.com/Product/Goods/91182412
95      http:/

In [6]:
print(book_list)

                            title    topic           isbn  \
0                        메타인지 학습법    가정 살림  9788950981891   
1                        김미경의 리부트     자기계발  9788901243665   
2               꽃 모티브 & 에징 손뜨개 60    건강 취미  9788969520524   
3                나는 3학년 2반 7번 애벌레      어린이  9788936414085   
4                     상처 받지 않는 영혼       인문  9788998075088   
..                            ...      ...            ...   
921                    100 인생 그림책      에세이  9791160944426   
922            가진 것이 없다면 운으로 승부하라     자기계발  9791190906012   
923          2020 한식조리기능사 필기 원큐패스  수험서 자격증  9788927770374   
924                내가 너의 첫문장이었을 때      에세이  9788901243283   
925  New 최태성 한국사능력검정시험 심화(1,2,3급)  수험서 자격증  9791129706126   

                                                review  
0    우리는 죽을 때까지 배워야 한다. 느껴야 한다. 그리고 공감해한다.나의 삶을 살면서...  
1     코로나가 오래 간다. 아무도 그렇게 생각하지도 못했던 시간들이 길어지고 있다. 대...  
2    재정가 도서라서 가성비는 높은 편이지만, 개인적으로는 좀 아쉬운 책이었어요.일단, ...  
3    아이가 관련 추천도서에 있길래 이것을 구매해 보았습니다.교실에

In [7]:
bookInfo = book_list.copy()
idx1 = bookInfo[bookInfo['topic']=='유아'].index
idx2 = bookInfo[bookInfo['topic']=='청소년'].index
idx3 = bookInfo[bookInfo['topic']=='예술'].index
idx4 = bookInfo[bookInfo['topic']=='어린이'].index
idx5 = bookInfo[bookInfo['topic']=='국어 외국어 사전'].index
idx6 = bookInfo[bookInfo['topic']=='수험서 자격증'].index
idx7 = bookInfo[bookInfo['topic']=='만화/라이트노벨'].index
idx8 = bookInfo[bookInfo['topic']=='건강 취미'].index

bookInfo = bookInfo.drop(idx1)
bookInfo = bookInfo.drop(idx2)
bookInfo = bookInfo.drop(idx3)
bookInfo = bookInfo.drop(idx4)
bookInfo = bookInfo.drop(idx5)
bookInfo = bookInfo.drop(idx6)
bookInfo = bookInfo.drop(idx7)
bookInfo = bookInfo.drop(idx8)

print(book_list.count())
print(bookInfo.count())

title     926
topic     926
isbn      926
review    926
dtype: int64
title     497
topic     497
isbn      497
review    497
dtype: int64


In [8]:
def getImg(isbnList):
    
    try:
        div = wd.Chrome()
        for t, isbn in enumerate(isbnList):
            searchterm = 'yes24 '+ isbn
            url = "https://www.google.com/search?q="+searchterm+"&source=lnms&tbm=isch"

            print(t)
            div.get(url)
            path = "C:/Users/HyunJu Kim/Desktop/졸작/yes24Img/"

            img = div.find_element_by_css_selector(".rg_i.Q4LuWd")
            img.click()
            div.implicitly_wait(3)
            src = div.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute('src')
            urlretrieve(src, path + isbn + ".jpg")

    except:
        print(isbn + ' 저장 실패')
    
    div.close()

In [9]:
if not os.path.exists('yes24Img'):
    os.mkdir('yes24Img')
    
isbnList = bookInfo['isbn'].tolist()
start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.submit(getImg, isbnList[:100])
    executor.submit(getImg, isbnList[100:200])
    executor.submit(getImg, isbnList[200:300])
    executor.submit(getImg, isbnList[300:400])
    executor.submit(getImg, isbnList[400:])
print("time :", time.time() - start)

0
0
0
0
0
1
1
1
1
1
2
2
2
2
3
3
3
3
2
4
4
4
4
3
5
5
5
5
6
6
6
4
6
7
7
7
7
8
5
8
8
8
9
9
9
9
10
6
10
10
11
10
7
11
11
12
11
12
8
12
13
12
13
13
14
9
13
14
14
10
15
14
15
15
11
16
15
16
16
12
16
17
17
17
17
13
18
18
18
18
19
14
19
19
20
19
15
20
20
20
21
21
16
22
21
21
22
17
22
23
22
23
23
18
24
23
24
25
24
19
24
25
26
25
20
26
25
27
26
21
26
27
27
28
22
28
27
28
29
23
29
29
28
30
24
30
30
29
31
31
25
31
30
32
32
26
31
32
33
33
34
32
27
33
33
34
35
28
34
36
35
34
29
35
37
36
35
30
38
36
36
31
37
39
37
40
38
37
38
41
32
39
39
38
42
33
40
39
43
40
41
34
44
40
41
42
35
42
41
45
43
44
43
42
46
36
45
43
44
37
47
44
48
45
46
38
49
45
46
39
47
50
46
48
47
51
40
47
49
48
52
48
41
49
53
50
42
49
54
50
43
51
50
44
51
52
51
55
53
52
52
56
54
45
53
55
53
57
46
54
58
56
54
55
59
57
55
47
56
58
56
57
60
48
59
57
58
61
49
60
61
62
50
58
59
63
62
51
59
60
63
64
52
60
61
64
61
53
65
62
65
66
62
63
54
66
67
63
68
64
55
67
64
65
56
68
69
65
69
66
57
66
70
70
67
67
58
71
68
71
68
72
59
69
72
69
60
73
70
73


In [11]:
bookInfo.to_excel("yes24_100.xlsx")
bookInfo.to_csv("yes24_100.csv", encoding='utf-8-sig')
print('저장')

저장


In [12]:
temp = pd.read_csv("yes24_100.csv")
print(temp['title'])

0                메타인지 학습법
1                김미경의 리부트
2             상처 받지 않는 영혼
3       세계의 끝과 하드보일드 원더랜드
4         더 해빙 The Having
              ...        
492            감정도 설계가 된다
493                   페인트
494            100 인생 그림책
495    가진 것이 없다면 운으로 승부하라
496        내가 너의 첫문장이었을 때
Name: title, Length: 497, dtype: object
